In [2]:
import snap
from itertools import combinations

In [3]:
def load_graph(name):
    
    if name == "w1":
        G = snap.TNEANet.Load(snap.TFIn("weight1-adjusted.graph"))
    if name == "w2":
        G = snap.TNEANet.Load(snap.TFIn("weight2-adjusted.graph"))
    if name == "w3":
        G = snap.TNEANet.Load(snap.TFIn("weight3-adjusted.graph"))
    
    return G
    

## Local features: 

#### Method 1: Common Neighbors  

$$ CN(x,y) = \sum\nolimits_{z \in |\Gamma(x) \cap \Gamma(y)|} w(x,z) + w(y,z)$$

#### Method 2: Jaccard Coefficients 

$$ JC(x,y) = \sum\nolimits_{z \in |\Gamma(x) \cap \Gamma(y)|} \frac{w(x,z) + w(y,z)}{{\sum\nolimits_{a \in \Gamma(x)}}w(a,x) + {\sum\nolimits_{b \in \Gamma(y)}}w(b,y)}$$

#### Method 3: Perferrential Attachment 
the probability that a new edge has node x as an endpoint is proportional to the its weights.


$$ PA(x,y) = \sum\nolimits_{a \in \Gamma(x)} w(a,x) * \sum\nolimits_{b \in \Gamma(y)} w(b,y)$$

#### Method 4:  Adamic-Adar Coefficient 
similar to Method 2, but it defines a higher importance to the common neighbors which have fewer neighbors.


$$ AA(x,y) = \sum\nolimits_{z \in |\Gamma(x) \cap \Gamma(y)|} \frac{w(x,z) + w(y,z)}{log (1+ {\sum\nolimits_{c \in \Gamma(z)}}w(a,x))}$$

In [4]:
def local_features(G,method):
    
    NIds = []
    for N in G.Nodes():
        NIds.append(N.GetId())
    pairs = combinations(NIds, 2) 
    
    if method == "CN":
        sim = common_neighbor(G,pairs)
    if method == "JC":
        sim = jaccard_coefficient(G,pairs)
    if method == "AA":
        sim = adamic_adar(G,pairs)
    if method == "PA":
        sim = prefer_attach(G,pairs)
    
    return sim  # a dictionary with all pairs as key and similarity score as value. 
    
    


In [5]:
def batch_pairs(G,batch_size):  #split all nodes into batches and apply combination on each batch ?
    NIds = []
    pairs = []
    for N in G.Nodes():
        NIds.append(N.GetId())
        
    for b in xrange(len(NIds)/batch_size):
        batch = NIds[b*batch_size:(b+1)*batch_size]
        pairs.append(combinations(batch, 2))
        
    if len(NIds)%batch_size != 0:
        batch_last = NIds[(b+1)*batch_size:]
        pairs.append(combinations(batch_last, 2))
    
    return pairs

In [6]:
def common_neighbor(G,pairs):
    
    CN = {}
    for uId, vId in pairs:
        CN[(uId,vId)] = 0
        Nbrs = snap.TIntV()
        cmn = snap.GetCmnNbrs(G,uId,vId,Nbrs)
        print "u",uId
        print "v",vId
        if cmn > 0:
            for zId in Nbrs:
                print "z",zId
                w1 = G.GetFltAttrDatE(G.GetEI(uId,zId), "weight") if (uId < zId) else G.GetFltAttrDatE(G.GetEI(zId,uId), "weight")
                w2 = G.GetFltAttrDatE(G.GetEI(vId,zId), "weight") if (vId < zId) else G.GetFltAttrDatE(G.GetEI(zId,vId), "weight")
                CN[(uId,vId)] += w1+w2
                
    return CN

In [7]:
def jaccard_coefficient(G,pairs):
    
    JC = {}
    for uId, vId in pairs:
        JC[(uId,vId)] = 0
        Nbrs = snap.TIntV()
        cmn = snap.GetCmnNbrs(G,uId,vId,Nbrs)
        
        if cmn > 0:            
            for zId in Nbrs:
                w1 = G.GetFltAttrDatE(G.GetEI(uId,zId), "weight") if (uId < zId) else G.GetFltAttrDatE(G.GetEI(zId,uId), "weight")
                w2 = G.GetFltAttrDatE(G.GetEI(vId,zId), "weight") if (vId < zId) else G.GetFltAttrDatE(G.GetEI(zId,vId), "weight")
                
                u = G.GetNI(uId)
                u_deg = u.GetDeg()
                sum_u = 0               
                
                for i in range(u_deg):  #compute edge weights between u and all its neighbors
                    aId = u.GetNbrNId(i)
                    ua = G.GetFltAttrDatE(G.GetEI(uId,aId), "weight") if (uId < aId) else G.GetFltAttrDatE(G.GetEI(aId,uId), "weight") 
                    sum_u +=ua
                
                v = G.GetNI(vId)
                v_deg = v.GetDeg()
                sum_v = 0 
                
                for i in range(v_deg):  #compute edge weights between v and all its neighbors
                    bId = v.GetNbrNId(i)
                    vb = G.GetFltAttrDatE(G.GetEI(vId,bId), "weight") if (vId < bId) else G.GetFltAttrDatE(G.GetEI(bId,vId), "weight") 
                    sum_v +=vb
                
                JC[(uId,vId)] += (w1+w2)*1.0 / (sum_u+sum_v)
                
    return JC

In [8]:
def adamic_adar(G,pairs):
    
    AA = {}
    for uId, vId in pairs:
        AA[(uId,vId)] = 0
        Nbrs = snap.TIntV()
        cmn = snap.GetCmnNbrs(G,uId,vId,Nbrs)
        
        if cmn > 0:            
            for zId in Nbrs:
                w1 = G.GetFltAttrDatE(G.GetEI(uId,zId), "weight") if (uId < zId) else G.GetFltAttrDatE(G.GetEI(zId,uId), "weight")
                w2 = G.GetFltAttrDatE(G.GetEI(vId,zId), "weight") if (vId < zId) else G.GetFltAttrDatE(G.GetEI(zId,vId), "weight")
                
                z = G.GetNI(zId)
                z_deg = z.GetDeg()
                sum_a = 0
                
                for i in range(z_deg):  #compute edge weights between common neighbor and its neigbors
                    cId = z.GetNbrNId(i)
                    a1 = G.GetFltAttrDatE(G.GetEI(cId,zId), "weight") if (cId < zId) else G.GetFltAttrDatE(G.GetEI(zId,cId), "weight") 
                    sum_a +=a1
                
                AA[(uId,vId)] += (w1+w2)*1.0 / np.log(1+sum_a)
    
    return AA

In [9]:
def prefer_attach(G,pairs):
   
    PA  = {}
    for uId, vId in pairs:
        PA[(uId,vId)] = 0        
        u = G.GetNI(uId)
        v = G.GetNI(vId)
        u_deg = u.GetDeg()
        v_deg = v.GetDeg()
        
        if (u_deg != 0) and (v_deg != 0):
            sum_u = 0               
            sum_v = 0
            for i in range(u_deg):  
                aId = u.GetNbrNId(i)
                ua = G.GetFltAttrDatE(G.GetEI(uId,aId), "weight") if (uId < aId) else G.GetFltAttrDatE(G.GetEI(aId,uId), "weight") 
                sum_u +=ua

            for i in range(v_deg):  
                bId = v.GetNbrNId(i)
                vb = G.GetFltAttrDatE(G.GetEI(vId,bId), "weight") if (vId < bId) else G.GetFltAttrDatE(G.GetEI(bId,vId), "weight") 
                sum_v +=vb

            PA[(uId,vId)] = sum_u * sum_v
        
    
    return PA